<a href="https://colab.research.google.com/github/KIQVinicius/Data-Science/blob/master/AnomaliasJob.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [311]:
import pandas as pd
from datetime import timedelta

In [312]:

from datetime import date

In [313]:
dataset = pd.read_csv("MIPS1.csv")
dataset.head()

,Unnamed: 0,JOBNAME,DATE,Valor_MIPS,Porcentagem,Chave
0,0,FG921RID,08/06/2020,0.01,0.3,FG921RID08/06/2020
1,1,FG921RII,08/06/2020,0.01,0.3,FG921RII08/06/2020
2,2,FG921RIS,08/06/2020,0.01,0.3,FG921RIS08/06/2020
3,3,FG9227,08/06/2020,0.11,0.3,FG922708/06/2020
4,4,FG9227A,08/06/2020,1.67,0.3,FG9227A08/06/2020


In [314]:
porcentagem = float(dataset['Porcentagem'][0])/100 + 1
dataset.columns = ['Index', 'JOBNAME', 'DATE', 'Valor_MIPS', 'Porcentagem', 'Chave']

In [315]:
dataset = dataset.drop(columns=["Porcentagem", "Chave"], axis=1)

In [316]:
dataset['DATE'] = pd.to_datetime(dataset['DATE'])
dataset = dataset.sort_values(['DATE'])

In [317]:
first_day = dataset['DATE'].min()
last_day = dataset['DATE'].max()

In [318]:
first_day

Timestamp('2019-01-11 00:00:00')

In [319]:
last_day

Timestamp('2020-12-06 00:00:00')

In [320]:
dt = last_day.date() - first_day.date()
dt.days

695

In [321]:
weeks = int(dt.days/7)
weeks

99

In [322]:
months = int(dt.days/30)
months

23

In [323]:
data = first_day
dataset['JOBNAME'].value_counts()

FG95BM09    262
TWSJSDEL    262
FZ90DPH1    262
DB2SDIST    262
FZ92DCV0    262
           ... 
FF016R4V      1
F6JP5DC4      1
FZ626EQ1      1
FG00PSI1      1
MSTRASFP      1
Name: JOBNAME, Length: 15631, dtype: int64

In [324]:
frequencia = pd.DataFrame(dataset['JOBNAME'].value_counts())
frequencia.reset_index(inplace=True)
frequencia.columns = ['JOBNAME', 'Frequencia']
frequencia

,JOBNAME,Frequencia
0,FG95BM09,262
1,TWSJSDEL,262
2,FZ90DPH1,262
3,DB2SDIST,262
4,FZ92DCV0,262
...,...,...
15626,FF016R4V,1
15627,F6JP5DC4,1
15628,FZ626EQ1,1
15629,FG00PSI1,1


In [325]:
frequencia = frequencia.query("Frequencia > 1")

In [326]:
dataset = dataset[dataset['JOBNAME'].isin(frequencia['JOBNAME'])]
dataset.head()

,Index,JOBNAME,DATE,Valor_MIPS
192478,192478,BD26B8L,2019-01-11,1.14
196089,196089,FS0245SI,2019-01-11,2.88
196088,196088,FS0245SF,2019-01-11,0.01
196087,196087,FS0245NL,2019-01-11,3.64
196086,196086,FS0245NG,2019-01-11,0.18


In [327]:
dataset = dataset.drop(columns=["Index"], axis=1)
dataset['JOBNAME'].value_counts()


PSNIMSTR    262
FG76BLR0    262
FX2242      262
FG92MPO1    262
FZ8130      262
           ... 
FG941G        2
FH08A0J       2
FG95AU        2
S1030540      2
PDJFN005      2
Name: JOBNAME, Length: 12923, dtype: int64

# Anomalia


In [328]:
def DetectarAnomalia(dataset, intervalo):
  if intervalo == "Mensal":
    periodo = months
    intervalo_de_dias = 30
  elif intervalo == "Semanal":
    periodo = weeks
    intervalo_de_dias = 7
  elif intervalo == "Quinzenal":
    periodo = int(dt.days/15)
    intervalo_de_dias = 15
  elif intervalo == "Trimestral":
    periodo = int(dt.days/90)
    intervalo_de_dias = 90
  elif intervalo == "Semestral":
    periodo = int(dt.days/180)
    intervalo_de_dias = 180
  elif intervalo == "Diario":
    periodo = int(dt.days)
    intervalo_de_dias = 1
  data = first_day
  tabela_anomalia = dataset.iloc[0:0]
  tabela_anomalia['Valor_MIPS_y'] = ""
  tabela_anomalia['Status'] = ""
  tabela_anomalia.columns = ['JOBNAME', 'DATE', 'Valor_MIPS_Atual', 'Valor_MIPS_Anterior', 'Status']
  dataset_semana_anterior = dataset.query("DATE >= " + " '"+ str(data)+ "' " + " and DATE < " + " '"+str(data+ timedelta(days = intervalo_de_dias)) + "' ")
  data = data+ timedelta(days = intervalo_de_dias)
  dataset_semana_anterior = dataset_semana_anterior[dataset_semana_anterior['Valor_MIPS']>0].groupby('JOBNAME')
  dataset_semana_anterior = pd.DataFrame(dataset_semana_anterior['Valor_MIPS'].mean())
  dataset_semana_anterior.reset_index(inplace =True)
  for i in range(0, periodo, 1):
    dataset_semana = dataset.query("DATE >= " + " '"+ str(data)+ "' " + " and DATE < " + " '"+str(data+ timedelta(days = intervalo_de_dias)) + "' ")
    #dataset = dataset.query("DATE >= " + " '"+ str(data+ timedelta(days = 30))+ "' ")
    print("-------------------------------------------------")
    #dataset_semana = dataset_semana[dataset_semana['Valor_MIPS']>0].groupby('JOBNAME')
    #dataset_semana = pd.DataFrame(dataset_semana['Valor_MIPS'].mean())
    dataset_semana_anterior = dataset_semana_anterior[dataset_semana_anterior['JOBNAME'].isin(dataset_semana['JOBNAME'])]

    df = pd.merge(dataset_semana, dataset_semana_anterior, on = "JOBNAME", how ='inner', suffixes=['_Atual', '_Anterior'])
    df = df[df['Valor_MIPS_Atual'] > df['Valor_MIPS_Anterior'] *porcentagem]
    df['Status'] = "Aumentou"
    tabela_anomalia = tabela_anomalia.append(df)
    df = df[df['Valor_MIPS_Atual'] < df['Valor_MIPS_Anterior'] *porcentagem]
    df['Status'] = "Diminuiu"
    tabela_anomalia = tabela_anomalia.append(df)

    print("Anomalias até " + str(data + timedelta(days = intervalo_de_dias) )[0:10]+": "  + str(tabela_anomalia.shape[0]))
    dataset_semana_anterior = dataset_semana[dataset_semana['Valor_MIPS']>0].groupby('JOBNAME')
    dataset_semana_anterior = pd.DataFrame(dataset_semana_anterior['Valor_MIPS'].mean())
    dataset_semana_anterior.reset_index(inplace=True)
    data = data+ timedelta(days = intervalo_de_dias)
  tabela_anomalia['Anomalia'] = intervalo
  return tabela_anomalia


In [329]:
tabela_mensal = DetectarAnomalia(dataset, "Mensal")

-------------------------------------------------
Anomalias até 2019-03-12: 3667
-------------------------------------------------
Anomalias até 2019-04-11: 8414
-------------------------------------------------
Anomalias até 2019-05-11: 12076
-------------------------------------------------
Anomalias até 2019-06-10: 17194
-------------------------------------------------
Anomalias até 2019-07-10: 20450
-------------------------------------------------
Anomalias até 2019-08-09: 24408
-------------------------------------------------
Anomalias até 2019-09-08: 28826
-------------------------------------------------
Anomalias até 2019-10-08: 33547
-------------------------------------------------
Anomalias até 2019-11-07: 66556
-------------------------------------------------
Anomalias até 2019-12-07: 115274
-------------------------------------------------
Anomalias até 2020-01-06: 144073
-------------------------------------------------
Anomalias até 2020-02-05: 190559
---------------

In [330]:
tabela_semanal = DetectarAnomalia(dataset, "Semanal")

-------------------------------------------------
Anomalias até 2019-01-25: 0
-------------------------------------------------
Anomalias até 2019-02-01: 0
-------------------------------------------------
Anomalias até 2019-02-08: 0
-------------------------------------------------
Anomalias até 2019-02-15: 0
-------------------------------------------------
Anomalias até 2019-02-22: 0
-------------------------------------------------
Anomalias até 2019-03-01: 0
-------------------------------------------------
Anomalias até 2019-03-08: 0
-------------------------------------------------
Anomalias até 2019-03-15: 0
-------------------------------------------------
Anomalias até 2019-03-22: 0
-------------------------------------------------
Anomalias até 2019-03-29: 0
-------------------------------------------------
Anomalias até 2019-04-05: 0
-------------------------------------------------
Anomalias até 2019-04-12: 0
-------------------------------------------------
Anomalias até 

In [331]:
tabela_quinzenal = DetectarAnomalia(dataset, "Quinzenal")

-------------------------------------------------
Anomalias até 2019-02-10: 0
-------------------------------------------------
Anomalias até 2019-02-25: 0
-------------------------------------------------
Anomalias até 2019-03-12: 1959
-------------------------------------------------
Anomalias até 2019-03-27: 4605
-------------------------------------------------
Anomalias até 2019-04-11: 5626
-------------------------------------------------
Anomalias até 2019-04-26: 8876
-------------------------------------------------
Anomalias até 2019-05-11: 9766
-------------------------------------------------
Anomalias até 2019-05-26: 13138
-------------------------------------------------
Anomalias até 2019-06-10: 13138
-------------------------------------------------
Anomalias até 2019-06-25: 13138
-------------------------------------------------
Anomalias até 2019-07-10: 13138
-------------------------------------------------
Anomalias até 2019-07-25: 13138
-----------------------------

In [332]:
tabela_trimestral = DetectarAnomalia(dataset, "Trimestral")

-------------------------------------------------
Anomalias até 2019-07-10: 15227
-------------------------------------------------
Anomalias até 2019-10-08: 29268
-------------------------------------------------
Anomalias até 2020-01-06: 152793
-------------------------------------------------
Anomalias até 2020-04-05: 288810
-------------------------------------------------
Anomalias até 2020-07-04: 362392
-------------------------------------------------
Anomalias até 2020-10-02: 388648
-------------------------------------------------
Anomalias até 2020-12-31: 418623


In [333]:
tabela_semestral = DetectarAnomalia(dataset, "Semestral")

-------------------------------------------------
Anomalias até 2020-01-06: 135035
-------------------------------------------------
Anomalias até 2020-07-04: 360098
-------------------------------------------------
Anomalias até 2020-12-31: 409026


In [334]:
#tabela_diaria = DetectarAnomalia(dataset, "Diario")


In [335]:
tabela_final = tabela_semanal.iloc[0:0]
tabela_final = tabela_final.append(tabela_mensal)
print(tabela_final.shape)
tabela_final = tabela_final.append(tabela_semanal)
print(tabela_final.shape)
tabela_final = tabela_final.append(tabela_quinzenal)
tabela_final = tabela_final.append(tabela_semestral)
#tabela_final = tabela_final.append(tabela_diaria)

(419505, 6)
(737968, 6)


In [336]:
tabela_diaria

,JOBNAME,DATE,Valor_MIPS_Atual,Valor_MIPS_Anterior,Status,DATA ANTERIOR,Anomalia
12,FTOMSTAT,2019-01-12,53.53,52.72,Aumentou,NaN,Diario
14,FS0205G0,2019-01-12,17.31,15.36,Aumentou,NaN,Diario
15,FS0205NL,2019-01-12,0.21,0.17,Aumentou,NaN,Diario
20,FR3451,2019-01-12,22.54,12.06,Aumentou,NaN,Diario
24,FS0205GA,2019-01-12,15.29,8.26,Aumentou,NaN,Diario
...,...,...,...,...,...,...,...
4614,FG95BM17,2020-12-06,0.18,0.07,Aumentou,NaN,Diario
4616,FG95BM15,2020-12-06,1.42,0.47,Aumentou,NaN,Diario
4617,FG95BR09,2020-12-06,0.02,0.01,Aumentou,NaN,Diario
4638,FG95BM13,2020-12-06,0.69,0.54,Aumentou,NaN,Diario


NameError: ignored